In [1]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_1431/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [4]:
playerClasses, ratingsHistory = epochsElo(matches)

In [5]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [8]:
singles_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1877-540,Wimbledon,Grass,32,G,1877-07-09,19,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1877-540,Wimbledon,Grass,32,G,1877-07-09,21,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1877-540,Wimbledon,Grass,32,G,1877-07-09,20,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1877-540,Wimbledon,Grass,32,G,1877-07-09,18,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1877-540,Wimbledon,Grass,32,G,1877-07-09,17,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876973,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,104,207987,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,716.0,31.0,966.0,12.0
876974,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,103,144932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,799.0,23.0,1117.0,8.0
876975,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,102,209955,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876976,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,205,209079,NaN,JE,...,NaN,NaN,NaN,NaN,NaN,NaN,1125.0,7.0,1548.0,2.0


In [12]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [26]:
max(ratingsHistory_df.index[ratingsHistory_df.index<'1880-07-08'])

Timestamp('1879-07-09 00:00:00')

In [ ]:
def winProb(winner_id,loser_id,tourney_date)

In [54]:
type(ratingsHistory_df.index[10])

pandas._libs.tslibs.timestamps.Timestamp

In [52]:
max(ratingsHistory_df.index[ratingsHistory_df.index<='1877-07-09'])

ValueError: max() arg is an empty sequence

In [27]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    if not type(tourney_date) == pandas._libs.tslibs.timestamps.Timestamp:
        

    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,winner_id,loser_id]
    wp = 1/(1 + 10**((loser_reting-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [45]:
get_recent_rating_wp(ratingsHistory_df,'1877-07-09',113987,114149)

ValueError: max() arg is an empty sequence

In [ ]:
matches

In [44]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [38]:
singles_matches.shape,matches.shape

((876978, 49), (876978, 4))

In [43]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=0)

KeyError: 'tourney_date'

In [14]:
ratingsHistory_df

,113953,113954,113955,113956,113957,113958,113959,113960,113961,113962,...,106408,106410,106415,106421,106422,106423,106425,106426,106432,204737
1878-07-09,1540.000000,1492.0,1508.000000,1492.0,1500.000000,1492.0,1500.000000,1492.000000,1516.0,1492.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1538.348263,1492.0,1500.734751,1492.0,1500.000000,1492.0,1492.000000,1508.736697,1516.0,1492.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1516.376828,1500.351618,1516.0,1492.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1508.000016,1500.351618,1516.0,1484.184174,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1882-07-08,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1508.000016,1500.351618,1516.0,1484.184174,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1484.485521,1500.351618,1516.0,1484.184174,...,1792.146721,1534.292549,2016.364557,2131.965382,1456.754994,2065.329039,1797.527094,1931.154564,2168.897866,1500.000000
2019-06-05,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1484.485521,1500.351618,1516.0,1484.184174,...,1892.984089,1467.016724,2099.617173,2416.346408,1456.754994,2043.309484,1797.527094,2242.359126,2287.878240,1500.000000
2020-06-04,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1484.485521,1500.351618,1516.0,1484.184174,...,1872.396425,1703.799734,2159.124813,2391.875211,1456.754994,2068.950029,1853.110312,2132.312132,2132.255175,1500.000000
2021-06-04,1538.348263,1492.0,1500.734751,1492.0,1491.815826,1492.0,1484.485521,1500.351618,1516.0,1484.184174,...,1978.313730,1668.742751,2058.187878,2417.352579,1456.754994,2043.249602,1834.960270,2162.949835,2196.547252,1500.000000
